# Amazon FBA Taxes 2021

In [1]:
# Set working directory
%cd "D:\Amazon FBA\businesstracker"

from amz_transactions import Transactions, Cashflows
from amz_charting import generate_cashflow_graph
import pandas as pd
import numpy as np


D:\Amazon FBA\businesstracker


In [2]:
# Connect to PSQL and create Transactions object
db = Transactions(configfile='psql_businessfinances.ini')

## Update Recieved Inventory in SQL from Inventory Reports

In [3]:
# df_inv = pd.read_csv("inventory_recieved.csv")
# df_inv["date"] = pd.to_datetime(df_inv["date"])
# for row in df_inv.itertuples():
#     query = f"INSERT INTO inventory_recieved VALUES ({row[0]},'{row[1]}','{row[2]}','{row[3]}','{row[4]}',{row[5]},'{row[6]}','{row[7]}',{row[8]}) RETURNING *"
#     db.execute_command(query)

## Retrieve datasets from PSQL

In [5]:
# Retrieve all transactions

query = f'''
        SELECT *
        FROM amz_transactions
        ORDER BY posted_date_time
        '''
df_tr = pd.read_sql(query,db.conn)
df_inv = pd.read_sql(f"SELECT * FROM inventory_recieved ORDER BY date",db.conn)
df_orders = pd.read_sql(f"SELECT DISTINCT order_id,posted_date_time FROM amz_transactions ORDER BY posted_date_time",db.conn)

In [6]:
## Helper function
def window_helper(group,op,col):
    group = group.sort_values('date')
    nam_col = op+col
    if op == "Sum":
        group[nam_col] = group[col].expanding(1).sum()
    elif op == "Avg":
        group[nam_col] = group[col].expanding(1).mean()
    return group

In [7]:
# Cleanup datasets

df_tr['date'] = df_tr["posted_date_time"].dt.date
df_tr['date'] = pd.to_datetime(df_tr['date'])

df_inv.rename(columns={"quantity":"inv_rec"}, inplace=True)
df_inv['date'] = pd.to_datetime(df_inv['date'])
# Convert random +1 and -1 inventory to 0 as they are not real entries
df_inv.loc[(df_inv["inv_rec"] >= -1) & (df_inv["inv_rec"] <= 1),"inv_rec"] = 0
df_inv["CPU"] = df_inv["cost"]/df_inv["inv_rec"]

In [8]:
### Merge in inventory orders
df_merged = df_tr.merge(df_inv[["date","inv_rec","CPU"]], how="outer", on="date")
df_merged["inv_rec"].fillna(0, inplace = True)
df_merged["CPU"].fillna(0, inplace = True)
# Cleanup CPU, first forward fill then set the initial NaNs to 0
df_merged["CPU"] = df_merged["CPU"].fillna(method="ffill").fillna(0)
# Remove duplicated inv_rec and CPU due to merge. Ensures inventory only added once
df_merged.loc[df_merged.duplicated(["date","inv_rec"]),"inv_rec"] = 0
df_merged.loc[df_merged.duplicated(["date","CPU"]),"CPU"] = 0

### Resort by dates to align everything
df_merged = df_merged.sort_values('date')
df_merged["quantity_purchased"] = df_merged["quantity_purchased"].fillna(0)

### Find the change in inventory by sales or refunds
df_merged["delta_inv"] = 0
df_merged.loc[df_merged["transaction_type"]=="Order","delta_inv"] = -1*df_merged["quantity_purchased"]
df_merged.loc[df_merged["transaction_type"]=="Refund","delta_inv"] = 1
# Remove duplicated due to merge. Ensures salers/refunds only added once per order
df_merged.loc[df_merged.duplicated(["order_id","delta_inv"]),"delta_inv"] = 0

### Clenaup index
df_merged = df_merged.reset_index(drop=True)

### Calculate current inventory based on inventory recieving and sales/refunds
df_merged["curr_inv"] = df_merged["inv_rec"] + df_merged["delta_inv"]
df_merged["curr_inv"] = df_merged["curr_inv"].expanding(1).sum()


In [10]:
## Calculate ACB

acb = np.zeros(len(df_merged))
for i in range(1,len(acb)):
    if max(0,df_merged.iloc[i]["inv_rec"]) > 0: # Increase ACB only if inventory increased
        curr_inv = df_merged.iloc[i]["curr_inv"]-df_merged.iloc[i]["inv_rec"]
        new_inv = df_merged.iloc[i]["inv_rec"]
        new_cost = new_inv*df_merged.iloc[i]["CPU"]
        curr_cost = curr_inv*acb[i-1]
        acb[i] = (new_cost+curr_cost)/(new_inv+curr_inv)
    else: # Else keep previous ACB
        acb[i] = acb[i-1]

df_merged["acb"] = acb

All data is prepared for analysis. Completed:

    1. Inventory recieving
    2. ACB calculation
    3. Inventory deltas for sales/refunds

## Analysis

In [25]:
start = '2021-01-01'
end = '2021-12-31'

df_tots = df_merged.groupby(["order_id","date"],as_index=False).sum()

df_tots.to_csv("test.csv")